In [1]:
# Import class files

import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [2]:
from queue import PriorityQueue
import math

example = open('example.txt', 'r').read()
puzzle = open('puzzle.txt', 'r').read()

input = puzzle

# Part 1

In [3]:
init_wires_info, logic_info = input.split('\n\n')

# Store name-to-wireval in a dictionary
wires = {}

remaining_wires = set()
z_wires = set()



# Initialise the starting wire values
for wire in init_wires_info.split('\n'):
    wires[wire[:3]] = int(wire[-1])


# Initialise needed logic
for logic in logic_info.split('\n'):
    wire1, gate, wire2, _, wire_out = logic.split(' ')

    # Append output wires to set
    if wire_out[0] == 'z':
        z_wires.add(wire_out)

    # Append wires to dict
    if wire_out not in wires:
        wires[wire_out] = wire1+gate+wire2
        remaining_wires.add(wire_out)

num_z_wires = len(z_wires)


# Iterate until we find all the z wire values
while(z_wires):

    # Iterate through all remaining wires to find, and find their value if we can
    to_remove = set()
    for wire in remaining_wires:
        logic_str = wires[wire]
        gate1, gate2, logic = logic_str[:3], logic_str[-3:], logic_str[3:-3]
        gate1_val, gate2_val = wires[gate1], wires[gate2]
        if gate1_val in [0,1] and gate2_val in [0,1]:
            match logic:
                case 'AND': val =  gate1_val and gate2_val
                case 'OR': val = gate1_val or gate2_val
                case 'XOR': val = gate1_val ^ gate2_val
            wires[wire] = val
            to_remove.add(wire)

            if wire[0] == 'z':
                z_wires.remove(wire)

    remaining_wires = remaining_wires - to_remove

def extract_z_wires(wires):
    '''
    Extracts the z wires from the wires dictionary and returns the binary number in decimal
    '''
    z_wires = []
    for wire in wires:
        if wire[0] == 'z':
            z_wires.append(wire)
    
    z_wires = sorted(z_wires, reverse=True)

    bin_str = ''
    for z_wire in z_wires:
        bin_str += str(wires[z_wire])
    
    return int(bin_str,2)

extract_z_wires(wires)

55544677167336

# Part 2

In [4]:
def get_num(letter, wires):
    '''
    Returns the number from binary of the variables given by [letter]00, [letter]01, ...
    '''
    letter_wires = []
    for wire in wires:
        if wire[0] == letter:
            letter_wires.append(wire)
    
    letter_wires = sorted(letter_wires, reverse=True)

    bin_str = ''
    for letter_wire in letter_wires:
        bin_str += str(wires[letter_wire])
    
    return int(bin_str,2), bin_str
    
def swap_wires(wires, wire1, wire2):
    '''
    Swaps the output in wires 1 and 2
    '''
    wires[wire1], wires[wire2] = wires[wire2], wires[wire1]
    return wires

def eval_wires(wires):
    '''
    Iterates through each of the wires until we find all of the z-wire values
    '''
    z_wires = set()
    remaining_wires = set()

    for wire in wires:
        if wire[0] == 'z':
            z_wires.add(wire)
        if not isinstance(wires[wire], int):
            remaining_wires.add(wire)

    while(z_wires):

        # Iterate through all remaining wires to find, and find their value if we can
        to_remove = set()
        for wire in remaining_wires:
            logic_str = wires[wire]
            gate1, gate2, logic = logic_str[:3], logic_str[-3:], logic_str[3:-3]
            gate1_val, gate2_val = wires[gate1], wires[gate2]
            if gate1_val in [0,1] and gate2_val in [0,1]:
                match logic:
                    case 'AND': val =  gate1_val and gate2_val
                    case 'OR': val = gate1_val or gate2_val
                    case 'XOR': val = gate1_val ^ gate2_val
                wires[wire] = val
                to_remove.add(wire)

                if wire[0] == 'z':
                    z_wires.remove(wire)

        remaining_wires = remaining_wires - to_remove
    
    return wires

def reset_wires():
    '''
    Resets wires to starting state and returns
    '''
    init_wires_info, logic_info = input.split('\n\n')

    # Store name-to-wireval in a dictionary
    wires = {}

    remaining_wires = set()
    z_wires = set()



    # Initialise the starting wire values
    for wire in init_wires_info.split('\n'):
        wires[wire[:3]] = int(wire[-1])


    # Initialise needed logic
    for logic in logic_info.split('\n'):
        wire1, gate, wire2, _, wire_out = logic.split(' ')

        # Append output wires to set
        if wire_out[0] == 'z':
            z_wires.add(wire_out)

        # Append wires to dict
        if wire_out not in wires:
            wires[wire_out] = wire1+gate+wire2
            remaining_wires.add(wire_out)
    return wires

In [5]:
# Going to try and go for a graph visualisation: Input all of the connections and we should hopefully see that most of it is consistent, but
# that some of it is not - wherever those happen, we should have a swapped gate!

# So to visualise this, we need a list of inputs into each gate, and gates leading into outputs

def generate_graph_code(wires, wires_to_swap=[]):
    '''Generates graphviz code given a set of wires to view the connections'''
    wires = reset_wires()
    if wires_to_swap:
        for swap in wires_to_swap:
            swap_wires(wires, swap[0], swap[1])

    connections = set()

    for wire in wires:
        logic = wires[wire]
        if not isinstance(wires[wire], int):
            in1, in2, gate, out = logic[:3], logic[-3:], logic[3:-3], wire
            for input_wire in [in1,in2]:
                connections.add((input_wire+' -> '+out, gate))


    con_str = ''

    for connection in connections:
        con_str += connection[0]+'; '

    x_str = ''
    y_str = ''
    z_str = ''

    for i in range(45):
        if i < 10:
            x_str += 'x0'+str(i)+' -> '
            y_str += 'y0'+str(i)+' -> '
            z_str += 'z0'+str(i)+' -> '
        else:
            x_str += 'x'+str(i)+' -> '
            y_str += 'y'+str(i)+' -> '
            z_str += 'z'+str(i)+' -> '

    z_str += 'z45'

    x_str = x_str[:-4]
    y_str = y_str[:-4]

    and_str = ''
    or_str = ''
    xor_str = ''

    for connection in connections:
        match connection[1]:
            case 'AND': and_str += connection[0][-3:]+'; '
            case 'OR': or_str += connection[0][-3:]+'; '
            case 'XOR': xor_str += connection[0][-3:]+'; '

    graphviz_code = f'''digraph G {{

    subgraph and_gate {{ 
        node[style=filled,color=orange];
        {and_str}
    }}

    subgraph or_gate {{
        node[style=filled, color=yellow];
        {or_str}
    }}

    subgraph xor_gate {{
        node[style=filled, color=green];
        {xor_str}
    }}

    {con_str}
    {x_str}
    {y_str}
    {z_str}
    }}
    '''

    return graphviz_code

print(generate_graph_code(wires))

digraph G {

    subgraph and_gate { 
        node[style=filled,color=orange];
        fph; cns; qgh; mdq; ssd; jtq; mfj; hsd; hwt; fhh; mfj; crb; qhw; jtq; mmm; bdj; mjb; psk; ctc; hnm; dvg; nng; thg; hkv; ftq; hwt; mck; ggw; qrs; pdt; skt; kmb; kmg; psk; fkw; bdj; dvg; fph; qmv; pbd; wfr; ggw; vkh; vth; kbg; fws; qmv; tct; qrs; mjb; nbj; pbd; mmm; fkw; drg; jhq; bjj; cnp; nsf; nvm; dbc; rbb; nkm; bvs; jhq; bjf; skj; rcn; ftq; mjs; hjd; hkv; gfv; cnp; qcb; skj; cns; bvs; tct; vkh; nhq; z32; frm; gfv; mdq; qtn; nvm; tbc; crb; vdn; knt; nkm; z26; rjj; hjf; ssd; rcn; bjf; nsf; hgk; rkn; hgk; cbq; psw; psw; tbc; nhq; vqk; mck; dgn; rgv; jvv; fhh; bbb; drg; gpv; hjf; ctc; pdt; hnm; khd; rnw; qhw; gth; hjd; z32; gwd; qcb; cbq; qnf; vdn; rgv; fws; rkn; qtn; khd; gpv; thg; gth; rbb; tkm; nbj; rnw; kbg; mtd; nng; gwd; mtd; frm; qsm; dbc; z26; wbb; qnf; dcc; vqk; bjj; bdr; jvv; vth; qsm; dcc; skt; rjj; bdr; bdg; qgh; bbb; kmg; hsd; mjs; kmb; bdg; wbb; tkm; knt; wfr; dgn; 
    }

    subgraph or

In [6]:
# Rules...

# - z gates are XOR
# - z gates have 1 XOR and 1 OR feeding into them
# - x and y gates feed into 1 AND and 1 XOR

# Swap z12 - kth
# Swap z26 - gsd
# Swap z32 - tbt
# Swap qnf - vpm


bad_gates = set(['z12','z26','z32','kth','gsd','qnf','vpm','tbt'])

In [7]:
print(generate_graph_code(wires, wires_to_swap=[['z12','kth'], ['z26','gsd'],['z32','tbt'], ['qnf','vpm']]))

digraph G {

    subgraph and_gate { 
        node[style=filled,color=orange];
        fph; cns; qgh; ssd; mdq; jtq; mfj; gsd; hsd; hwt; fhh; mfj; crb; qhw; jtq; mmm; bdj; mjb; psk; ctc; hnm; dvg; nng; thg; hkv; ftq; hwt; mck; ggw; qrs; pdt; skt; kmb; kmg; psk; fkw; bdj; dvg; fph; qmv; pbd; wfr; ggw; vkh; vth; kbg; fws; qmv; tct; qrs; mjb; nbj; pbd; mmm; fkw; drg; jhq; bjj; cnp; nsf; tbt; nvm; dbc; rbb; nkm; bvs; jhq; bjf; skj; rcn; ftq; mjs; hjd; hkv; gfv; cnp; qcb; skj; cns; bvs; tct; vkh; nhq; frm; gfv; mdq; qtn; nvm; tbc; crb; vdn; knt; nkm; rjj; hjf; ssd; rcn; bjf; nsf; hgk; rkn; hgk; cbq; psw; psw; tbc; nhq; vqk; mck; dgn; rgv; jvv; fhh; bbb; drg; gpv; hjf; ctc; pdt; hnm; khd; rnw; qhw; gth; hjd; gwd; qcb; cbq; vdn; rgv; fws; rkn; qtn; khd; gpv; thg; gth; rbb; vpm; tbt; tkm; nbj; rnw; kbg; mtd; nng; gwd; mtd; frm; qsm; dbc; wbb; dcc; vqk; bjj; bdr; jvv; vth; qsm; dcc; skt; rjj; bdr; bdg; qgh; bbb; kmg; hsd; mjs; kmb; bdg; wbb; tkm; vpm; knt; gsd; wfr; dgn; 
    }

    subgraph or

In [8]:
wire_keys = list(wires.keys())
wire_keys

orig_wires = reset_wires()
x_num = get_num('x', wires)
y_num = get_num('y', wires)
target = x_num[0] + y_num[0]

x_num, y_num, target

((33884112699961, '111101101000101000010100000000100001000111001'),
 (21587482918575, '100111010001000111010010000000011111010101111'),
 55471595618536)

In [9]:
bad_gates = list(bad_gates)
','.join(sorted(bad_gates))

'gsd,kth,qnf,tbt,vpm,z12,z26,z32'